In [20]:
from tensorflow import keras
import pandas as pd
import numpy as np
import sklearn

In [18]:
dataset = pd.read_csv("infarto.csv", delimiter=";")
dataset.drop(columns=["id", "casado"], inplace=True)
dataset.dropna(inplace=True)
dataset

,genero,edad,hipertensión,enfermedad_coronaria,trabajo,residencia,nivel_glucosa,bmi,uso_tabaco,infarto
0,Masculino,67.0,0,1,Privado,Urbana,228.69,36.6,Ex fumador,1
2,Masculino,80.0,0,1,Privado,Rural,105.92,32.5,Nunca,1
3,Femenino,49.0,0,0,Privado,Urbana,171.23,34.4,Fumador,1
4,Femenino,79.0,1,0,Autonomo,Rural,174.12,24.0,Nunca,1
5,Masculino,81.0,0,0,Privado,Urbana,186.21,29.0,Ex fumador,1
...,...,...,...,...,...,...,...,...,...,...
5104,Femenino,13.0,0,0,Menor de edad,Rural,103.08,18.6,NSNC,0
5106,Femenino,81.0,0,0,Autonomo,Urbana,125.20,40.0,Nunca,0
5107,Femenino,35.0,0,0,Autonomo,Rural,82.99,30.6,Nunca,0
5108,Masculino,51.0,0,0,Privado,Rural,166.29,25.6,Ex fumador,0


In [27]:
import sklearn.preprocessing


datasetNp = dataset.to_numpy()
encoder = sklearn.preprocessing.OneHotEncoder(handle_unknown='ignore')
categorical_cols = ["genero", "edad", "hipertensión", "enfermedad_coronaria",
                   "trabajador", "residencia", "uso_tabaco"]

encoder.fit(dataset)

OneHotEncoder(handle_unknown='ignore')

In [28]:
encodedData = encoder.transform(dataset)

In [31]:
encoder.categories_

[array(['Femenino', 'Masculino', 'Other'], dtype=object),
 array([8.00e-02, 1.60e-01, 2.40e-01, 3.20e-01, 4.00e-01, 4.80e-01,
        5.60e-01, 6.40e-01, 7.20e-01, 8.00e-01, 8.80e-01, 1.00e+00,
        1.08e+00, 1.16e+00, 1.24e+00, 1.32e+00, 1.40e+00, 1.48e+00,
        1.56e+00, 1.64e+00, 1.72e+00, 1.80e+00, 1.88e+00, 2.00e+00,
        3.00e+00, 4.00e+00, 5.00e+00, 6.00e+00, 7.00e+00, 8.00e+00,
        9.00e+00, 1.00e+01, 1.10e+01, 1.20e+01, 1.30e+01, 1.40e+01,
        1.50e+01, 1.60e+01, 1.70e+01, 1.80e+01, 1.90e+01, 2.00e+01,
        2.10e+01, 2.20e+01, 2.30e+01, 2.40e+01, 2.50e+01, 2.60e+01,
        2.70e+01, 2.80e+01, 2.90e+01, 3.00e+01, 3.10e+01, 3.20e+01,
        3.30e+01, 3.40e+01, 3.50e+01, 3.60e+01, 3.70e+01, 3.80e+01,
        3.90e+01, 4.00e+01, 4.10e+01, 4.20e+01, 4.30e+01, 4.40e+01,
        4.50e+01, 4.60e+01, 4.70e+01, 4.80e+01, 4.90e+01, 5.00e+01,
        5.10e+01, 5.20e+01, 5.30e+01, 5.40e+01, 5.50e+01, 5.60e+01,
        5.70e+01, 5.80e+01, 5.90e+01, 6.00e+01, 6.10e+01, 